# Aides aux Entreprises - PDF Reports

## Imports

In [1]:
import json
import os
import datetime
import random

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import numpy as np

from fpdf import FPDF

## Functions

In [2]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [3]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [4]:
def format_number_to_str(x) :
    if x == x :
        res = str(format(int(round(float(x),0)),",")).replace(","," ").replace(".",",")
    else :
        res = 'NaN'
    return res

In [5]:
def format_number_to_million_str(x) :
    if x == x :
        res = str(round(float(x)/1000000,1)).replace('.',',')
    else :
        res = 'NaN'
    return res

In [6]:
def format_str_to_int(x) :
    if x != 'None' :
        res = int(float(x))
    else : 
        res = None
    return res

In [7]:
def shorten_long_naf_libelle(libelle) :
    if libelle == "Production et distribution d'eau ; assainissement, gestion des déchets et dépollution" :
        return "Prod./dist. eau, assainissement, déchets, dépollution"
    else :
        return libelle

In [8]:
months_fr_dict = {
    '01':'janvier',
    '02':'février',
    '03':'mars',
    '04':'avril',
    '05':'mai',
    '06':'juin',
    '07':'juillet',
    '08':'août',
    '09':'septembre',
    '10':'octobre',
    '11':'novembre',
    '12':'décembre'
}

# yearmonth should be in '2020-01' format

def yearmonth_to_fr(yearmonth) :
    month = yearmonth[-2:]
    year = yearmonth[:4]
    
    return months_fr_dict[month] + ' ' + year

In [9]:
def make_chart(aide, kpi, maille, chart_type, x_col, y_col, x_label, y_label, others_y_val, title, legend_title=None, show=False, sort_by_y_col_inc=False, sort_by_y_col_dec=False, stack_order=None, dep=None, reg=None, text_col=None, stack_col=None, color_col='color') :
    
    data_path = './data/{}-{}-{}.csv'.format(aide, kpi, maille)
    
    if os.path.isfile(data_path) :
    
        #Loading data
        if maille == 'regional' :
            dtype={'reg':'str'}
        elif maille == 'departemental' :
            dtype={'dep':'str'}
        else :
            dtype={}
        
        df = pd.read_csv(data_path, dtype=dtype)

        #Filtering if departemental or regional
        if maille == 'departemental' :
            df['dep'] = df['dep'].apply(str)
            df = df[df['dep'] == dep]
            if len(df) == 0 :
                fig = None
                print('No data to plot for {}-{}-{}-{}'.format(aide, kpi, maille, dep))
        elif maille == 'regional' :
            df['reg'] = df['reg'].apply(str)
            df = df[df['reg'] == reg]
            if len(df) == 0 :
                fig = None
                print('No data to plot for {}-{}-{}-{}'.format(aide, kpi, maille, reg))

        if len(df) > 0 :

            #Sorting by x_col and putting "Others" (others_y_val) in the end
            if sort_by_y_col_inc == True :
                df_plot = pd.concat([df[df[y_col] != others_y_val].sort_values(by=y_col, ascending=True), df[df[y_col] == others_y_val]]).reset_index(drop=True)
            elif sort_by_y_col_dec == True :
                df_plot = pd.concat([df[df[y_col] != others_y_val].sort_values(by=y_col, ascending=False), df[df[y_col] == others_y_val]]).reset_index(drop=True)
            else : #sort by decreasing x_col value by default
                df_plot = pd.concat([df[df[y_col] != others_y_val].sort_values(by=x_col, ascending=False), df[df[y_col] == others_y_val]]).reset_index(drop=True)    
            
            if stack_order is not None :
                df_stack_list = []
                for stack_val in stack_order :
                    df_stack = df_plot[df_plot[stack_col] == stack_val]
                    df_stack_list += [df_stack]
                
                df_plot = pd.concat(df_stack_list).reset_index(drop=True)
            
            #Parameters for plotting
            if text_col is not None :
                text = text_col
            else :
                text = x_col + '_fr'

            if chart_type == 'hbar' :

                df_plot[y_col] = df_plot[y_col].apply(lambda x : x+'   ')
                color_discrete_map = {key:value for (key,value) in zip(df_plot[y_col], df_plot[color_col])}

                fig = px.bar(df_plot, x=x_col, y=y_col,
                         labels={y_col:y_label,x_col:x_label}, 
                         height=550,
                         width=800,
                         color=y_col,
                         color_discrete_map = color_discrete_map,
                         orientation='h',
                         text=text,
                         title=title)
                fig.update_layout(showlegend=False)
                fig.layout['xaxis'].tickfont = dict(color = 'rgba(0,0,0,0)')
                fig.layout['xaxis'].showgrid = False
                fig.for_each_trace(lambda t: t.update(textfont_color='white', insidetextanchor='middle', textposition='inside'))

            elif chart_type == 'sthbar' :

                color_discrete_map = {key:value for (key,value) in zip(df_plot[stack_col], df_plot[color_col])}

                fig = px.bar(df_plot, x=x_col, y=y_col,
                         labels={y_col:y_label,x_col:x_label, stack_col:legend_title}, 
                         height=550,
                         width=800,
                         color=stack_col,
                         color_discrete_map = color_discrete_map,
                         orientation='h',
                         title=title)
                
                fig.update_yaxes(type='category', categoryorder='category descending')
                fig.layout['yaxis'].tickmode = 'array'
                fig.layout['yaxis'].tickvals = sorted(list(set(df_plot['mois'])), reverse=True)
                fig.layout['yaxis'].ticktext = [yearmonth_to_fr(yearmonth) + '   ' for yearmonth in sorted(list(set(df_plot['mois'])), reverse=True)]

            elif chart_type == 'dpie' :

                color_discrete_map = {key:value for (key,value) in zip(df_plot[y_col], df_plot[color_col])}

                fig = px.pie(df_plot, values=x_col, names=y_col,
                     labels={y_col:y_label,x_col:x_label}, 
                     height=400,
                     width=600,
                     hole=0.5,
                     color=y_col,
                     color_discrete_map = color_discrete_map,
                     title=title)

                fig.for_each_trace(lambda t: t.update(textfont_color='white', textposition='inside'))

            #Center the main title
            fig.update_layout(title={'x':0.5, 'xanchor': 'center'})

            if show == True :
                fig.show()
    
    else :
        fig = None
        print('File not found: {}'.format(data_path))
    
    return fig

## Parameters

In [10]:
data_dir_path = './data/'
img_dir_path = './img/'
reports_dir_path = './reports/'

In [11]:
#Taxonomies

taxo_nafsec_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/sections-naf-minify.json'
taxo_jurcat_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/categories-juridiques-minify.json'
taxo_effcla_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/classes-effectifs-minify.json'
taxo_nace17_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/sections-nace17-minify.json'

taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'

In [12]:
mailles_list = ['national', 'regional', 'departemental']

In [13]:
#DATA PROCESSING: KPIs to process
aides_charts_kpi_dict = {
    'aides': ['kpi_top_10_naf', 'kpi_categorie_juridique', 'kpi_classe_effectif'],
    'pge':['kpi_top_10_naf'],
    'report':['kpi_top_10_naf'],
    'cpsti':['kpi_top_10_naf'],
    'activite-partielle':['nace17']
}

#TODO: macro KPIs
aides_macro_kpi_dict = {
    'aides': ['montant', 'nombre', 'nombre_siren'],
    'pge':['montant', 'nombre'],
    'report':['montant', 'nombre'],
    'cpsti':['montant', 'nombre'],
    'activite-partielle':['nombre_salaries_concernes', 'nombre_etablissements_concernes', 'nombre_heures_demandees']
}

In [14]:
#PLOTTING: parameters for graphs
#As of now, we consider these graphs are the same for national, regional and departemental

graph_params_list = [
    
    #Fonds de solidarité
    dict(aide='aides',
         kpi='kpi_top_10_naf',
         x_col='montant',
         y_col='libelle_section_naf',
         x_label='',
         y_label='Section NAF',
         others_y_val='Autres sections NAF',
         title='Top 10 des aides du fonds de solidarité<br />ventilées par code section NAF (en M€)',
         chart_type='hbar'
        ),
    dict(aide='aides',
         kpi='kpi_categorie_juridique',
         maille='national',
         x_col='montant',
         y_col='libelle_cat_juridique',
         x_label='',
         y_label="Catégorie juridique",
         others_y_val="Autres Catégories Juridiques",
         title='Top 3 des aides du fonds de solidarité<br />ventilées par catégories juridiques (en M€)',
         chart_type='dpie'
        ),
    dict(aide='aides',
         kpi='kpi_classe_effectif',
         x_col='montant',
         y_col='libelle_classe_effectif',
         x_label='',
         y_label="Classe d'effectif",
         sort_by_y_col_dec=True,
         others_y_val="Autres Classes d'Effectifs",
         title="Aides du fonds de solidarité<br />ventilées par classes d'effectifs (en M€)",
         chart_type='hbar'
        ),
    
    #Prêts garantis par l'Etat
    dict(aide='pge',
         kpi='kpi_top_10_naf',
         maille='national',
         x_col='montant',
         y_col='libelle_section_naf',
         x_label='',
         y_label='Section NAF',
         others_y_val='Autres sections NAF',
         title="Top 10 des prêts garantis par l'Etat<br />ventilés par code section NAF (en M€)",
         chart_type='hbar'
        ),
    
    #Reports d'échéances fiscales
    dict(aide='report',
         kpi='kpi_top_10_naf',
         x_col='montant',
         y_col='libelle_section_naf',
         x_label='',
         y_label='Section NAF',
         others_y_val='Autres sections NAF',
         title="Top 10 des reports d'échéances fiscales<br />ventilés par code section NAF (en M€)",
         chart_type='hbar'
        ),
    
    #Aides artisans/commerçants
    dict(aide='cpsti',
         kpi='kpi_top_10_naf',
         x_col='montant',
         y_col='libelle_section_naf',
         x_label='',
         y_label='Section NAF',
         others_y_val='Autres sections NAF',
         title="Top 10 des aides CPSTI<br />ventilés par code section NAF (en M€)",
         chart_type='hbar'
        ),
    
    #Activité partielle
    dict(aide='activite-partielle',
         kpi='nace17',
         x_col='nombre_salaries_concernes',
         y_col='mois',
         stack_col='libelle',
         x_label='',
         y_label='Mois',
         sort_by_y_col_inc=True,
         others_y_val='Autres sections NACE 17',
         stack_order=['Commerce', "Act. scientif. et tech. ; services admin. et de soutien", 'Hébergement et restauration', 'Construction', "Fabrication d'autres produits indus.", 'Autres sections NACE 17'],
         title="Nombre de salariés concernés par l'activité partielle<br />ventilés par code section NACE 17",
         legend_title=' ',
         chart_type='sthbar'
        ),
]

## Loading taxonomies data

In [15]:
taxo_nafsec_df = pd.DataFrame(import_json_to_dict(taxo_nafsec_url))
taxo_jurcat_df = pd.DataFrame(import_json_to_dict(taxo_jurcat_url))
taxo_effcla_df = pd.DataFrame(import_json_to_dict(taxo_effcla_url))
taxo_nace17_df = pd.DataFrame(import_json_to_dict(taxo_nace17_url))

taxo_dep_df = pd.DataFrame(import_json_to_dict(taxo_dep_url))
taxo_reg_df = pd.DataFrame(import_json_to_dict(taxo_reg_url))

## Data pre-processing

In [16]:
%%time
#Charts data processing (one CSV file per aide type, KPI and maille)

mkdir_ifnotexist(data_dir_path)

for aide in aides_charts_kpi_dict.keys() :
    for maille in mailles_list :
        
        #Condition to drop when activite-partielle data is back to "normal" folder
        if aide != 'activite-partielle' : 
            #url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}/{}-maille-{}-minify.json".format(aide, aide, maille)
            url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}/{}-maille-{}.json".format(aide, aide, maille)
        else :
            #url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}3/{}-3-maille-{}-minify.json".format(aide, aide, maille)
            url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}3/{}-3-maille-{}.json".format(aide, aide, maille)
            
        data = import_json_to_dict(url)
        
        if len(data) > 0 :
    
            for kpi in aides_charts_kpi_dict[aide] :

                df_list = []

                if maille == 'departemental' :
                    for item in data :
                        df_dep = pd.DataFrame(item[kpi])
                        df_dep['dep'] = item['dep']
                        df_list += [df_dep]

                elif maille == 'regional' :
                    for item in data :
                        df_reg = pd.DataFrame(item[kpi])
                        df_reg['reg'] = item['reg']
                        df_list += [df_reg]

                else :
                    df_list = [pd.DataFrame(data[0][kpi])]

                #Concatenating data in a single dataframe
                df = pd.concat(df_list).reset_index(drop=True)

                #Adding color information for plotting
                if kpi == 'kpi_top_10_naf' :
                    df = df.merge(taxo_nafsec_df[['code_section', 'color_section']].rename(columns={'code_section':'section_naf'}), on='section_naf', how='left')
                    df.drop('section_naf', 1, inplace=True)
                    df.rename(columns={'color_section':'color'}, inplace=True)

                elif kpi == 'kpi_categorie_juridique' :
                    df = df.merge(taxo_jurcat_df[['code', 'color']].rename(columns={'code':'code_cat_juridique'}), on='code_cat_juridique', how='left')
                    df.drop('code_cat_juridique', 1, inplace=True)

                elif kpi == 'kpi_classe_effectif' :
                    df = df.merge(taxo_effcla_df[['denomination', 'color']].rename(columns={'denomination':'classe_effectif'}), on='classe_effectif', how='left')
                    df.drop('classe_effectif', 1, inplace=True)

                elif kpi == 'nace17' : #Only activite-partielle as of now
                    df = df.merge(taxo_nace17_df[['code_section_nace17', 'color_section']], on='code_section_nace17', how='left')
                    df.drop('code_section_nace17', 1, inplace=True)
                    df.rename(columns={'color_section':'color'}, inplace=True)
                    
                    row_df_list = []

                    for row in df.iterrows() :

                        row_df = pd.DataFrame(eval(str(row[1]['data'])))

                        row_df['libelle'] = row[1]['libelle']
                        if 'reg' in df.columns:
                            row_df['reg'] = row[1]['reg']
                            row_df['reg'] = row_df['reg'].apply(lambda x : str(x).zfill(2))
                        if 'dep' in df.columns:
                            row_df['dep'] = row[1]['dep']
                            row_df['dep'] = row_df['dep'].apply(lambda x : str(x).zfill(2))
                        row_df['color'] = row[1]['color']

                        row_df_list += [row_df]

                    df = pd.concat(row_df_list).reset_index(drop=True)
                    
                    for col in ['nombre_demandes_deposees', 'nombre_heures_demandees', 'nombre_salaries_concernes'] :
                        df[col] = df[col].apply(format_str_to_int)

                else :
                    print('Unknown KPI, could not assign colors.')

                df['color'].fillna('#AAAAAA', inplace=True)

                #Creating labels for plotting (depending on existing columns)

                if 'montant' in df.columns :
                    df['montant_fr'] = df['montant'].apply(lambda x: format_number_to_million_str(x) + " M€")

                if 'nombre' in df.columns :
                    df['nombre_fr'] = df['nombre'].apply(lambda x: format_number_to_str(x) + " aides")

                if 'nombre_siren' in df.columns :
                    df['nombre_siren_fr'] = df['nombre_siren'].apply(lambda x: format_number_to_str(x) + " entreprises")

                if 'nombre_demandes_deposees' in df.columns :
                    df['nombre_demandes_deposees_fr'] = df['nombre_demandes_deposees'].apply(lambda x: format_number_to_str(x) + " demandes")

                if 'nombre_etablissements_concernes' in df.columns :
                    df['nombre_etablissements_concernes_fr'] = df['nombre_etablissements_concernes'].apply(lambda x: format_number_to_str(x) + " entreprises")

                if 'nombre_heures_demandees' in df.columns :
                    df['nombre_heures_demandees_fr'] = df['nombre_heures_demandees'].apply(lambda x: format_number_to_str(x) + " heures")

                if 'nombre_salaries_concernes' in df.columns :
                    df['nombre_salaries_concernes_fr'] = df['nombre_salaries_concernes'].apply(lambda x: format_number_to_str(x) + " salariés")
                    
                #NAF: shorten NAF section that is too long and dropping '0' values (does not make sense with "top 10")
                if kpi == 'kpi_top_10_naf' :
                    df['libelle_section_naf'] = df['libelle_section_naf'].apply(shorten_long_naf_libelle)
                    df = df[df['montant'].apply(float) > 0]

                df.to_csv(os.path.join(data_dir_path, '{}-{}-{}.csv'.format(aide, kpi, maille)), index=False)
                
        else :
            print('Empty data for {}-{}'.format(aide,maille))

<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:50: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:54: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:50: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:54: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:50: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:54: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
<timed exec>:58: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


<timed exec>:58: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


Empty data for activite-partielle-departemental
CPU times: user 1.06 s, sys: 8.7 ms, total: 1.07 s
Wall time: 2.81 s


In [17]:
print(url)

https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/activite-partielle3/activite-partielle-3-maille-departemental.json


## Making the graphs

In [18]:
#Get departements and regions lists

taxo_dep_df = pd.DataFrame(import_json_to_dict(taxo_dep_url))
dep_list = list(taxo_dep_df['dep'])
print('{} departements.'.format(len(dep_list)))

taxo_reg_df = pd.DataFrame(import_json_to_dict(taxo_reg_url))
reg_list = list(taxo_reg_df['reg'])
print('{} regions.'.format(len(reg_list)))

101 departements.
18 regions.


In [19]:
#Building the folders structures
mkdir_ifnotexist(img_dir_path)
mkdir_ifnotexist(os.path.join(img_dir_path, 'national'))

#Regions subfolders
mkdir_ifnotexist(os.path.join(img_dir_path, 'regional'))
for reg in reg_list :
    mkdir_ifnotexist(os.path.join(img_dir_path, 'regional', reg))
    
#Departements subfolders
mkdir_ifnotexist(os.path.join(img_dir_path, 'departemental'))
for dep in dep_list :
    mkdir_ifnotexist(os.path.join(img_dir_path, 'departemental', dep))

In [20]:
%%time
#Generating all graphs and saving them in appropriate folders
for maille in mailles_list :
    for param in graph_params_list :
        graph_param = param.copy()
        graph_param['maille'] = maille
        
        if maille == 'national' :
            img_path = os.path.join(img_dir_path, maille, '{}-{}-{}-{}-{}.png'.format(param['aide'],
                                                                                      param['kpi'],
                                                                                      param['x_col'],
                                                                                      maille,
                                                                                      param['chart_type']))

            fig = make_chart(**graph_param)
            
            if fig is not None :
                fig.write_image(img_path)
            
        elif maille == 'regional' :
            for reg in reg_list :
                graph_param['reg'] = reg
                img_path = os.path.join(img_dir_path, maille, reg, '{}-{}-{}-{}-{}.png'.format(param['aide'],
                                                                                               param['kpi'],
                                                                                               param['x_col'],
                                                                                               maille,
                                                                                               reg,
                                                                                               param['chart_type']))
                fig = make_chart(**graph_param) 
                
                if fig is not None :
                    fig.write_image(img_path)
                
        elif maille == 'departemental' :
            for dep in dep_list :
                graph_param['dep'] = dep
                img_path = os.path.join(img_dir_path, maille, dep, '{}-{}-{}-{}-{}.png'.format(param['aide'],
                                                                                               param['kpi'],
                                                                                               param['x_col'],
                                                                                               maille,
                                                                                               dep,
                                                                                               param['chart_type']))
                fig = make_chart(**graph_param) 
                
                if fig is not None :
                    fig.write_image(img_path)
                    
        print('{}-{}-{} done.'.format(maille, param['aide'], param['kpi']))

national-aides-kpi_top_10_naf done.
national-aides-kpi_categorie_juridique done.


national-aides-kpi_classe_effectif done.
national-pge-kpi_top_10_naf done.


national-report-kpi_top_10_naf done.
national-cpsti-kpi_top_10_naf done.


national-activite-partielle-nace17 done.


regional-aides-kpi_top_10_naf done.


...

departemental-aides-kpi_categorie_juridique done.


departemental-aides-kpi_classe_effectif done.


departemental-pge-kpi_top_10_naf done.


departemental-report-kpi_top_10_naf done.


No data to plot for cpsti-kpi_top_10_naf-departemental-976
departemental-cpsti-kpi_top_10_naf done.
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activite-partielle-nace17-departemental.csv
File not found: ./data/activi

## Building the reports

In [21]:
mkdir_ifnotexist(reports_dir_path)

In [22]:
global title_header
title_header = ''
global subtitle_header
subtitle_header = ''

In [23]:
class PDF(FPDF):
    def header(self):
        if (self.page_no() != 1 ):
            # Logo
            self.image(os.path.join(img_dir_path, 'LogoRF.png'), 10, 8, 33)
            self.image(os.path.join(img_dir_path, 'logo_Francerelance_1321294.54.png'), pdf.w-45, 7, 45)
            # Arial bold 15
            self.cell(50)
            self.set_font('Arial', 'B', 16)
            self.cell(80, 15, title_header, 0, 1, 'A')

            # Move to the right
            # Title
            self.set_font('Arial', 'I', 9)
            self.cell(50)
            self.cell(50, 10, subtitle_header+' - aides-entreprises.data.gouv.fr', 0, 1, 'A', link='https://aides-entreprises.data.gouv.fr/')
            # Line break  
            pdf.line(40, 38, 170, 38)
            self.ln(7)
        
    # Page footer
    def footer(self):
        if (self.page_no() != 1 ):
            # Position at 1.5 cm from bottom
            self.set_y(-15)
            # Arial italic 8
            self.set_font('Arial', 'I', 8)
            # Page number
            
            self.cell(0, 10, 'Direction Interministérielle du Numérique (DINUM) - Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')         

In [24]:
def format_val(val,add):
    return '{:,}'.format(int(float(val))).replace(',', ' ')+add

In [25]:
def get_kpis(aide,maille,geo):
    #url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}/{}-maille-{}-minify.json".format(aide, aide, maille)
    url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}/{}-maille-{}.json".format(aide, aide, maille)
    
    data = import_json_to_dict(url)
    if(maille == 'national'):
        if(aide == 'aides'):
            return format_val(data[0]['montant'],' euros'),format_val(data[0]['nombre'],''),format_val(data[0]['nombre_siren'],'')
        else:
            return format_val(data[0]['montant'],' euros'),format_val(data[0]['nombre'],'')

    elif(maille == 'regional'):
        for reg in data:
            if(reg['reg'] == geo):
                if(aide == 'aides'):
                    return format_val(reg['montant'],' euros'),format_val(reg['nombre'],''),format_val(reg['nombre_siren'],'')
                else:
                    return format_val(reg['montant'],' euros'),format_val(reg['nombre'],'')

    elif(maille == 'departemental'):
        for dep in data:
            if(dep['dep'] == geo):
                if(aide == 'aides'):
                    return format_val(dep['montant'],' euros'),format_val(dep['nombre'],''),format_val(dep['nombre_siren'],'')
                else:
                    return format_val(dep['montant'],' euros'),format_val(dep['nombre'],'')


In [26]:
def get_actpart_kpis(aide,maille,geo):
    #url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}3/{}-3-maille-{}-minify.json".format(aide, aide, maille)
    url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/{}3/{}-3-maille-{}.json".format(aide, aide, maille)
    
    data = import_json_to_dict(url)
    if(maille == 'national'):
        return format_val(data[0]['nombre_demandes_deposees'],''), format_val(data[0]['nombre_salaries_concernes'],''),format_val(data[0]['nombre_heures_demandees'],'')

    elif(maille == 'regional'):
        for reg in data:
            if(reg['reg'] == geo):
                return format_val(reg['nombre_demandes_deposees'],''), format_val(reg['nombre_salaries_concernes'],''),format_val(reg['nombre_heures_demandees'],'')

    elif(maille == 'departemental'):
        for dep in data:
            if(dep['dep'] == geo):
                return format_val(dep['nombre_demandes_deposees'],''), format_val(dep['nombre_salaries_concernes'],''),format_val(dep['nombre_heures_demandees'],'')

In [27]:
def find_last_update(aide):
    if aide != 'activite-partielle' :
        url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/"+aide+"/last_update_data.txt"
    else :
        url = "https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/"+aide+"3/last_update_data.txt"
    u2 = urllib.request.urlopen(url)
    return u2.readlines()[0].decode("utf-8")

In [28]:
def add_page_report(pdf, aide, dep, aideurl, phrase1, phrase2, phrase3, phrase_nombre, source_dict):

    pdf.set_fill_color(83,101,125);
    pdf.rect(20,41,180,14,'F')
    
    pdf.image(os.path.join(img_dir_path, 'information.png'), x=10, y=44.5, w=7)
    
    
    pdf.set_font('Arial', 'I', 7)
    pdf.set_text_color(255,255,255)
    pdf.cell(10)
    pdf.cell(10, 3, phrase1, ln=1)
    pdf.ln(0)
    pdf.cell(10)    
    pdf.cell(10, 3, phrase2, ln=1)
    pdf.ln(0)
    pdf.cell(10)
    pdf.cell(10, 3, phrase3, ln=1)
    pdf.ln(0)
    pdf.cell(10)
    pdf.set_font('Arial', 'IU', 7)
    pdf.cell(10, 3, source_dict['name'], ln=1, link=source_dict['url'])
    pdf.ln(2)
       
    pdf.set_text_color(0,0,0)


    pdf.set_font('Arial', 'BI', 14)
    pdf.cell(65, 10, "Niveau National", ln=1)
    
    
    pdf.set_font('Arial', '', 14)

    hbar_w = 100
    dpie_w = 90

    pdf.image(os.path.join(img_dir_path, 'national', aide+'-kpi_top_10_naf-montant-national-hbar.png'), x=pdf.w/3-hbar_w/2, y=64, w=hbar_w)

    kpis_nat = get_kpis(aide,'national','national')
    pdf.ln(10)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Montant total : "+kpis_nat[0], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Nombre "+phrase_nombre+kpis_nat[1], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    if(aide == 'aides'):
        pdf.cell(0, 10, "Nombre d'entreprises : "+kpis_nat[2], 0, 0)


    pdf.ln(27)

    reg = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['reg']
    libelle_reg = taxo_reg_df[taxo_reg_df['reg'] == reg].iloc[0]['libelle']
    libelle_dep = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
    
    pdf.set_font('Arial', 'BI', 14)
    pdf.cell(65, 10, "Niveau Régional : "+libelle_reg, ln=1)
    
    if aide == 'cpsti' and reg == '06' :
        pdf.set_font('Arial', 'I', 12)
        pdf.ln(10)
        pdf.cell(0,0,"Absence de données à l'échelle de la région.", ln=1)
        pdf.ln(15)
    else :
        pdf.set_font('Arial', '', 14)

        hbar_w = 100
        dpie_w = 90

        kpis_reg = get_kpis(aide,'regional', reg)

        pdf.image(os.path.join(img_dir_path, 'regional/'+reg+'/'+aide+'-kpi_top_10_naf-montant-regional-'+reg+'.png'), x=pdf.w/3-hbar_w/2, y=142, w=hbar_w)
        pdf.ln(10)
        pdf.cell((pdf.w/3)*1.55)
        pdf.cell(0, 10, "Montant total : "+kpis_reg[0], 0, 0)
        pdf.ln(15)
        pdf.cell((pdf.w/3)*1.55)
        pdf.cell(0, 10, "Nombre "+phrase_nombre+kpis_reg[1], 0, 0)
        pdf.ln(15)
        pdf.cell((pdf.w/3)*1.55)
        if(aide == 'aides'):
            pdf.cell(0, 10, "Nombre d'entreprises : "+kpis_reg[2], 0, 0)

        pdf.ln(25)

    pdf.set_font('Arial', 'BI', 14)
    pdf.cell(65, 15, "Niveau Départemental : "+libelle_dep, ln=1)
        
    if aide == 'cpsti' and dep == '976' :
        pdf.ln(10)
        pdf.set_font('Arial', 'I', 12)
        pdf.cell(0,0,"Absence de données à l'échelle du département.", ln=1)
        pdf.ln(15)
    else :
        pdf.set_font('Arial', '', 14)

        hbar_w = 100
        dpie_w = 90

        pdf.image(os.path.join(img_dir_path, 'departemental/'+dep+'/'+aide+'-kpi_top_10_naf-montant-departemental-'+dep+'.png'), x=pdf.w/3-hbar_w/2, y=219, w=hbar_w)

        kpis_dep = get_kpis(aide,'departemental', dep)
        pdf.ln(8)
        pdf.cell((pdf.w/3)*1.55)
        pdf.cell(0, 10, "Montant total : "+kpis_dep[0], 0, 0)
        pdf.ln(15)
        pdf.cell((pdf.w/3)*1.55)
        pdf.cell(0, 10, "Nombre "+phrase_nombre+kpis_dep[1], 0, 0)
        pdf.ln(15)
        pdf.cell((pdf.w/3)*1.55)
        if(aide == 'aides'):
            pdf.cell(0, 10, "Nombre d'entreprises : "+kpis_dep[2], 0, 0)
    
    pdf.link(10,57,120,8, 'http://aides-entreprises.data.gouv.fr')
    pdf.link(10,134,120,8, 'https://aides-entreprises.data.gouv.fr/'+aideurl+'?datasetid=regions&field=code&value='+reg)
    pdf.link(10,212,120,8, 'https://aides-entreprises.data.gouv.fr/'+aideurl+'?datasetid=departements&field=code&value='+dep)
    
    
    return pdf

In [29]:
def add_actpart_page_report(pdf, dep, aideurl, phrase1, phrase2, phrase3, source_dict):
    
    aide = 'activite-partielle'
    
    pdf.set_fill_color(83,101,125);
    pdf.rect(20,41,180,14,'F')
    
    pdf.image(os.path.join(img_dir_path, 'information.png'), x=10, y=44.5, w=7)
    
    
    pdf.set_font('Arial', 'I', 7)
    pdf.set_text_color(255,255,255)
    pdf.cell(10)
    pdf.cell(10, 3, phrase1, ln=1)
    pdf.ln(0)
    pdf.cell(10)    
    pdf.cell(10, 3, phrase2, ln=1)
    pdf.ln(0)
    pdf.cell(10)
    pdf.cell(10, 3, phrase3, ln=1)
    pdf.ln(0)
    pdf.cell(10)
    pdf.set_font('Arial', 'IU', 7)
    pdf.cell(10, 3, source_dict['name'], ln=1, link=source_dict['url'])
    pdf.ln(2)
       
    pdf.set_text_color(0,0,0)


    pdf.set_font('Arial', 'BI', 14)
    pdf.cell(65, 10, "Niveau National", ln=1)
    
    
    pdf.set_font('Arial', '', 14)

    hbar_w = 100
    dpie_w = 90

    pdf.image(os.path.join(img_dir_path, 'national', aide+'-nace17-nombre_salaries_concernes-national-sthbar.png'), x=pdf.w/3-hbar_w/2, y=64, w=hbar_w)

    kpis_nat = get_actpart_kpis(aide,'national','national')
    pdf.ln(10)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Nombre de demandes : "+kpis_nat[0], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Nombre de salariés : "+kpis_nat[1], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Heures demandées : "+kpis_nat[2], 0, 0)


    pdf.ln(27)

    reg = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['reg']
    libelle_reg = taxo_reg_df[taxo_reg_df['reg'] == reg].iloc[0]['libelle']
    libelle_dep = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
    
    pdf.set_font('Arial', 'BI', 14)
    pdf.cell(65, 10, "Niveau Régional : "+libelle_reg, ln=1)

    pdf.set_font('Arial', '', 14)

    hbar_w = 100
    dpie_w = 90

    kpis_reg = get_actpart_kpis(aide,'regional', reg)

    pdf.image(os.path.join(img_dir_path, 'regional/'+reg+'/'+aide+'-nace17-nombre_salaries_concernes-regional-'+reg+'.png'), x=pdf.w/3-hbar_w/2, y=142, w=hbar_w)
    pdf.ln(10)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Nombre de demandes : "+kpis_reg[0], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Nombre de salariés : "+kpis_reg[1], 0, 0)
    pdf.ln(15)
    pdf.cell((pdf.w/3)*1.55)
    pdf.cell(0, 10, "Heures demandées : "+kpis_reg[2], 0, 0)

    pdf.ln(25)
    
    #Pas de données départementales pour l'instant pour l'activité partielle
    
    pdf.link(10,57,120,8, 'http://aides-entreprises.data.gouv.fr')
    pdf.link(10,134,120,8, 'https://aides-entreprises.data.gouv.fr/'+aideurl+'?datasetid=regions&field=code&value='+reg)
    
    return pdf

In [30]:
%%time

mkdir_ifnotexist(os.path.join(reports_dir_path, 'pdf'))

for dep in taxo_dep_df.dep.unique():
    
    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()

    #Logos
    pdf.image(os.path.join(img_dir_path, 'LogoRF.png'), 10, 8, 33)
    pdf.image(os.path.join(img_dir_path, 'logo_Francerelance_1321294.54.png'), pdf.w-45, 7, 45)
    
    # Arial bold 15
    pdf.set_font('Arial', 'B', 28)
    # Move to the right
    pdf.cell(50)
    pdf.ln(70)
    
    # Title
    pdf.cell(10)
    pdf.cell(50, 10, 'SUIVI DES AIDES VERSÉES', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, 'AUX ENTREPRISES DANS LE CADRE', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, 'DE LA CRISE SANITAIRE COVID-19', 0, 1, 'A')

    pdf.set_font('Arial', 'I', 20)
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, 'Données pour le département : "'+taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']+'"', 0, 1, 'A')
    pdf.ln(110)

    pdf.set_font('Arial', 'I', 8)

    pdf.cell(10)
    pdf.cell(50, 10, 'Données issues du tableau de bord des aides aux entreprises consultable sur https://aides-entreprises.data.gouv.fr', 0, 1, 'A')
    
    #FONDS DE SOLIDARITÉ
    
    phrase1 = "Depuis le début de la crise sanitaire de la COVID-19, l'État et les Régions ont mis en place un fonds de solidarité pour prévenir la cessation d'activité des petites" # 
    phrase2 = "entreprises, micro-entrepreneurs, indépendants et professions libérales, particulièrement touchés par les conséquences économiques de la COVID-19."
    phrase3 = ""
    source_dict = {'name':'Les données utilisées proviennent de la DGFIP et sont disponibles en open data sur data.gouv.fr', 'url':'https://www.data.gouv.fr/fr/datasets/aides-aux-entreprises-dans-le-cadre-de-lepidemie-de-covid-19-en-france/'}
    
    title_header = 'FONDS DE SOLIDARITÉ'
    subtitle_header = find_last_update('aides')
    pdf.add_page()
    pdf = add_page_report(pdf, 'aides', dep, 'fds', phrase1, phrase2, phrase3, 'd\'aides : ', source_dict)

    #PGE
    
    phrase1 = "Le Gouvernement met en oeuvre un dispositif exceptionnel de garanties permettant de soutenir le financement bancaire des entreprises, à hauteur de 300 milliards"
    phrase2 = "d'euros."
    phrase3 = "Les entreprises peuvent désormais contracter un prêt jusqu'au 30 juin 2021 au lieu du 31 décembre 2020."
    source_dict = {'name':'Les données utilisées proviennent de la DG Trésor et sont disponibles en open data sur data.gouv.fr', 'url':'https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-prets-garantis-par-letat-dans-le-cadre-de-lepidemie-de-covid-19/'}
    
    title_header = 'PRÊTS GARANTIS PAR L\'ÉTAT'
    subtitle_header = find_last_update('pge')
    pdf.add_page()
    pdf = add_page_report(pdf, 'pge', dep, 'pge', phrase1, phrase2, phrase3, 'de prêts : ', source_dict)

    #REPORTS D'ECHEANCES FISCALES
    
    phrase1 = "Le Gouvernement a annoncé le report des principales échéances fiscales des professionnels afin de tenir compte de leurs difficultés liées à la crise sanitaire "
    phrase2 = "de la COVID-19."
    phrase3 = ""
    source_dict = {'name':'Les données utilisées proviennent de la DGFIP et sont disponibles en open data sur data.gouv.fr', 'url':'https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-reports-decheances-fiscales-accordes-dans-le-cadre-de-lepidemie-de-covid-19/'}
    
    title_header = 'REPORTS D\'ÉCHÉANCES FISCALES'
    subtitle_header = find_last_update('report')
    pdf.add_page()
    pdf = add_page_report(pdf, 'report', dep, 'reports', phrase1, phrase2, phrase3, 'de reports : ', source_dict)

    #CPSTI
    
    phrase1 = "Le Conseil de la protection sociale des travailleurs indépendants (CPSTI) met en place une aide financière exceptionnelle à destination de tous les commerçants"
    phrase2 = "et les artisans qui sont en activité au 15 mars 2020 et ont été immatriculés avant le 1er janvier 2019."
    phrase3 = ""
    source_dict = {'name':"Les données utilisées proviennent de l'URSSAF et sont disponibles en open data sur data.gouv.fr", 'url':'https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-aides-exceptionnelles-aux-artisans-et-commercants-dans-le-cadre-de-lepidemie-de-covid-19/'}
    
    title_header = 'AIDES AUX ARTISANS, COMMERÇANTS'
    subtitle_header = find_last_update('cpsti')
    pdf.add_page()
    pdf = add_page_report(pdf, 'cpsti' , dep,'cpsti', phrase1, phrase2, phrase3, 'd\'aides : ', source_dict)
    
    #ACTIVITE PARTIELLE
    
    phrase1 = "L'activité partielle permet à l'employeur en difficulté de faire prendre en charge tout ou partie du coût de la rémunération de ses salariés. L'activité partielle"
    phrase2 = "s'adresse à tous les salariés qui subissent une baisse de rémunération imputable soit à une réduction de l'horaire de travail pratiqué dans l'établissement ou"
    phrase3 = "partie de l'établissement en deçà de la durée légale de travail, soit à une fermeture temporaire de tout ou partie de l'établissement."
    source_dict = {'name':'Les données utilisées proviennent de la DARES', 'url':'https://dares.travail-emploi.gouv.fr/dares-etudes-et-statistiques/tableaux-de-bord/le-marche-du-travail-pendant-le-covid-19/tableaux-de-bord-hebdomadaires'}
    
    title_header = 'ACTIVITÉ PARTIELLE'
    subtitle_header = find_last_update('activite-partielle')
    pdf.add_page()
    pdf = add_actpart_page_report(pdf, dep, 'activitepartielle', phrase1, phrase2, phrase3, source_dict)
    

    pdf.output(os.path.join(reports_dir_path, 'pdf', 'Suivi_aides_entreprises_'+taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']+'.pdf'), 'F')
    
    print(str(datetime.datetime.today()) + ' - ' + dep + ' done.')

2022-10-19 02:48:42.162689 - 01 done.


2022-10-19 02:48:49.080724 - 02 done.


2022-10-19 02:48:55.954106 - 03 done.


2022-10-19 02:49:02.817430 - 04 done.


2022-10-19 02:49:10.884753 - 05 done.


...

2022-10-19 02:59:26.844445 - 971 done.


2022-10-19 02:59:33.507553 - 972 done.


2022-10-19 02:59:40.164490 - 973 done.


2022-10-19 02:59:46.836831 - 974 done.


2022-10-19 02:59:52.605469 - 976 done.
CPU times: user 10min 20s, sys: 5.44 s, total: 10min 25s
Wall time: 11min 17s


## Compress to ZIP

In [31]:
from zipfile import ZipFile

In [32]:
today_date = str(datetime.datetime.today())[:10]
today_date

'2022-10-19'

In [33]:
my_zip_date = ZipFile(os.path.join(reports_dir_path, 'Suivi_aides_entreprises_{}.zip'.format(today_date)), 'w')
my_zip_latest = ZipFile(os.path.join(reports_dir_path, 'Suivi_aides_entreprises_latest.zip'), 'w')

for filename in os.listdir(os.path.join(reports_dir_path, 'pdf')) :
    if filename.endswith('.pdf') :
        my_zip_date.write(os.path.join(reports_dir_path, 'pdf', filename))
        my_zip_latest.write(os.path.join(reports_dir_path, 'pdf', filename))
        
my_zip_date.close()
my_zip_latest.close()